## Neuronetwork Classifier

In [41]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.layers import Conv1D
from keras.layers import Dropout
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [42]:
df = pd.read_csv('./treatmenetds.csv')

In [87]:
df.head(20)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,target_description
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1,take antibiotics
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1,take antibiotics
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1,take antibiotics
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1,take antibiotics
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1,take antibiotics
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1,take antibiotics
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1,take antibiotics
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1,take antibiotics
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1,take antibiotics
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1,take antibiotics


In [43]:
X = df.drop(columns=['target','target_description'])
Y = df.target

In [44]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [70]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=13, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [71]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)

In [72]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 66.85% (13.84%)


## Gradient Boosted Trees

In [75]:
#Splitting dataset into Training (80%) and testing data (20%) using train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [83]:
#Create an XGB classifier and instance of the same
from xgboost import XGBClassifier
clf = XGBClassifier()
clf

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [84]:
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)

In [85]:
y_pred = clf.predict(X_test)

In [86]:
#classification accuracy
from sklearn import metrics
print(f'Accuracy: {metrics.accuracy_score(y_test, y_pred)}')

Accuracy: 0.8524590163934426
